In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from matplotlib.ticker import PercentFormatter
import kaplanmeier as km
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import seaborn as sns
from statannot import add_stat_annotation
from itertools import combinations 

In [2]:
# Read in sample z-score data from 'data_mRNA_median_all_sample_Zscores.txt' into pandas dataframe
z_scores = pd.read_csv('../GSE62564/GSE62564_zscores.csv',index_col=False)
clinical = pd.read_csv('../GSE62564/GSE62564_clinical_data.csv')

/Users/brandonbergs/anaconda2/envs/v3_8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
z_scores

,RefSeqID,symbol,SEQC_NB001,SEQC_NB002,SEQC_NB003,SEQC_NB004,SEQC_NB005,SEQC_NB006,SEQC_NB007,SEQC_NB008,...,SEQC_NB489,SEQC_NB490,SEQC_NB491,SEQC_NB492,SEQC_NB493,SEQC_NB494,SEQC_NB495,SEQC_NB496,SEQC_NB497,SEQC_NB498
0,NM_000014,A2M,-0.609942,-0.639064,-0.366959,0.220934,-0.170388,-0.477985,1.071831,-0.540778,...,-0.500736,0.928043,-0.686387,0.155410,-0.521667,1.004488,0.799726,0.078966,0.733293,1.566899
1,NM_000015,NAT2,-0.591986,NaN,-0.582482,0.621822,1.157453,-0.451167,0.052499,NaN,...,NaN,NaN,NaN,-0.274927,-1.485279,-0.477948,-0.522872,-1.048135,-1.112065,-0.546198
2,NM_000016,ACADM,0.660874,-0.052203,-1.877854,1.283744,-1.562124,-0.138116,0.678057,0.611474,...,-3.095671,-1.302237,0.368770,-2.092637,1.303074,0.300040,0.871361,0.327962,0.169023,0.381657
3,NM_000017,ACADS,0.132708,-1.153375,-1.395983,-0.042074,-1.600765,0.563141,-0.490768,-2.055980,...,-2.079459,-0.267726,-0.527290,-1.444243,-0.014683,-0.259900,-0.668159,-0.488160,-1.119462,0.007491
4,NM_000018,ACADVL,-0.430734,0.117632,-1.203080,-1.263323,-1.079505,0.312264,-0.480164,-1.856486,...,-1.590798,-0.923491,-0.767477,-1.166008,0.326166,0.565593,0.363239,0.738599,0.876077,-0.158868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43822,XR_133572,NaN,0.027817,-0.762377,-0.786605,0.066953,-1.873122,0.560825,0.818010,-1.470570,...,-0.057912,-1.968168,1.078923,0.111681,0.454596,0.704327,0.696872,0.995059,1.185152,0.676372
43823,XR_133573,NaN,1.072617,-0.422108,0.598442,-0.301125,-0.888478,1.008223,0.735035,-0.260147,...,0.508680,-1.598765,0.789673,0.358427,0.147683,0.658933,-0.381130,1.246286,1.739974,0.413065
43824,XR_133574,NaN,-0.010509,0.002905,-0.177230,0.229032,-0.503006,0.560558,1.051138,-0.980172,...,-0.338202,-2.559229,0.918911,-0.474261,0.411084,0.357426,0.692784,1.112461,1.587711,0.073810
43825,XR_133575,NaN,NaN,NaN,NaN,-0.804499,-0.454408,NaN,NaN,-0.929218,...,-0.264047,0.342047,1.217272,NaN,0.423005,-2.106396,NaN,-1.937915,-2.099832,0.613366


In [9]:
clinical

,SampID,GSM,Sex,Age,EFS Time,EFS Bin,OS Time,OS Bin,MYCN Amplification,High Risk,INSS Stage,Progression,Death from Disease
0,SEQC_NB001,GSM1528894,M,987.0,593,1,1362,1,0.0,1,4,1,1
1,SEQC_NB002,GSM1528895,M,1808.0,2016,1,2836,1,0.0,1,4,1,1
2,SEQC_NB003,GSM1528896,F,625.0,840,1,1191,1,1.0,1,4,1,1
3,SEQC_NB004,GSM1528897,F,335.0,2046,0,2046,0,1.0,1,2,0,0
4,SEQC_NB005,GSM1528898,F,536.0,212,1,220,1,1.0,1,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,SEQC_NB494,GSM1529389,M,56.0,4828,0,4828,0,0.0,0,1,0,0
494,SEQC_NB495,GSM1529390,M,163.0,2467,0,2467,0,0.0,0,1,0,0
495,SEQC_NB496,GSM1529391,M,132.0,105,1,5780,0,0.0,0,1,1,0
496,SEQC_NB497,GSM1529392,F,379.0,4883,0,4883,0,0.0,0,1,0,0


In [16]:
#This function stratifies samples based on the expression of a gene of interest
#Inputs: gene of interest, lower z-score threshold, upper z-score threshold
#Outputs: 2-D array containing lists of samples in each stratified group
def stratifyDataOnGeneExpression(gene,lowerThresh,upperThresh,z_scores=z_scores):
    #Obtain the z-scores for the gene of interest
    gene_z_scores = z_scores.loc[z_scores['symbol']==gene].iloc[:,2:].transpose()
    #Find samples for which the z-scores are < lowerThresh
    lower_expression_samples = gene_z_scores.loc[gene_z_scores.iloc[:,0] < lowerThresh].index
    #Find samples for which the z-scores are > upperThresh
    higher_expression_samples = gene_z_scores.loc[gene_z_scores.iloc[:,0] > upperThresh].index
    return [lower_expression_samples, higher_expression_samples]

In [ ]:
#This function stratifies samples based on the expression of a gene of interest
#Inputs: gene of interest, lower z-score threshold, upper z-score threshold
#Outputs: 2-D array containing lists of samples in each stratified group
#Stratified data into three groups (includes intermediate group), unlike original function
def stratifyDataOnGeneExpression2(gene,lowerThresh,upperThresh):
    #Obtain the z-scores for gene of interest
    gene_z_scores = z_scores.loc[z_scores['symbol']==gene].iloc[:,2:].transpose()
    #Find samples for which the z-scores are < lowerThresh
    lower_expression_samples = gene_z_scores.loc[gene_z_scores.iloc[:,0] < lowerThresh].index
    #Find samples for which z-scores are between thresholds
    intermediate_z_scores = gene_z_scores.loc[gene_z_scores.iloc[:,0] > lowerThresh]
    intermediate_expression_samples = intermediate_z_scores.loc[intermediate_z_scores.iloc[:,0] < upperThresh].index
    #Find samples for which the z-scores are > upperThresh
    higher_expression_samples = gene_z_scores.loc[gene_z_scores.iloc[:,0] > upperThresh].index
    return [lower_expression_samples, intermediate_expression_samples, higher_expression_samples]

In [115]:
#This function stratifies samples based on information in the clinical dataframe
#Inputs: Column you want to stratify data on,
#list of the categories within the stratifyingColumn you want to stratify on
#Output: 2-D array containing lists of samples in each stratified group
def stratifyClinicalData(stratifyingColumn,categories):
    #Statify samples based on categories in stratifyingColumn
    groups = []
    for category in categories:
        groups.append(clinical.loc[clinical[stratifyingColumn]==category]['SampID'])
    
    return groups

In [ ]:
#This function stratifies samples based on age at diagnosis
#Inputs: age threshold (in days) you want to stratify data on
#Output: 2-D array containing lists of samples in each stratified group
def stratifyDataOnAge(thresh):
    young = clinical.loc[clinical['Age']<thresh]['SampID']
    old = clinical.loc[clinical['Age']>=thresh]['SampID']
    return [young,old]

In [121]:
#This function compares the expression of a gene of interest between multiple groups
#Inputs: gene of interest, 2-D array containing lists of samples in each groups (output of functions above),
#label of stratifying variable and list of labels for each category (for labelling graphs)
#Outputs: boxplot and violin plot of gene expression compared between groups, summary statistics
def compareExpression(gene,groups,stratifyingLabel,categoryLabels, testType):
    #Find z-scores for specified gene for each group, and store in list
    gene_expression_values = []
    for group in groups:
        group_columns = ['RefSeqID','symbol'] + list(set(group).intersection(z_scores.columns))
        group_z_scores = z_scores[group_columns]
        group_gene_expression_df = group_z_scores.loc[group_z_scores['symbol']==gene]
        group_gene_expression_values = group_gene_expression_df.iloc[0,2:].values.astype(float)
        gene_expression_values.append(group_gene_expression_values)
    
    #Update category labels to include sample size
    updated_categoryLabels = []
    for i in range(len(categoryLabels)):
        updated_categoryLabels.append(categoryLabels[i] + ' (n = ' + str(len(gene_expression_values[i])) + ')')
    
    #Make dataframe of gene expression values
    gene_expression_values_df = pd.DataFrame(index=range(max([len(i) for i in gene_expression_values])))
    for i in range(len(categoryLabels)):
        gene_expression_values_df[categoryLabels[i]] = pd.Series(gene_expression_values[i])
    
    #Violin plot comparing expression between groups
    ax1 = sns.violinplot(data=gene_expression_values)
    ax1.set_ylabel('z-scores')
    ax1.set_xticklabels(updated_categoryLabels)
    ax1.set_title(gene + ' Expression stratified by ' + stratifyingLabel)
    ax1, test_results = add_stat_annotation(ax1, data=gene_expression_values_df,
                    box_pairs=list(combinations(categoryLabels,2)),
                    test=testType, text_format='simple', loc='inside', verbose=2, comparisons_correction=None)
    plt.show()
    
    #Box plot comparing expression between groups
    ax2 = sns.boxplot(data=gene_expression_values,palette='Greys')
    for patch in ax2.artists:
        r, g, b, a = patch.get_facecolor()
        patch.set_facecolor((r, g, b, 0.7))
    ax2.set_ylabel('z-scores')
    ax2.set_xticklabels(updated_categoryLabels)
    ax2.set_title(gene + ' Expression stratified by ' + stratifyingLabel)
    ax2.set_ylim(-5,5)
    ax2, test_results = add_stat_annotation(ax2, data=gene_expression_values_df,
                    box_pairs=list(combinations(categoryLabels,2)),
                    test=testType, text_format='simple', loc='inside', verbose=2, comparisons_correction=None)
    plt.show()
    
    #Print median, mean, and 95% CI of z-scores for each group
    for i in range(len(gene_expression_values)):
        print(categoryLabels[i] + ": median = " + str(np.nanmedian(gene_expression_values[i]))
         + ", mean = " + str(np.nanmean(gene_expression_values[i])))
        

    
    
    

In [ ]:
#This function compares the expression of a gene of interest between multiple groups
#Inputs: gene of interest, 2-D array containing lists of samples in each groups (output of functions above),
#label of stratifying variable and list of labels for each category (for labelling graphs)
#Outputs: boxplot and violin plot of gene expression compared between groups, summary statistics
def compareExpression2(gene,groups,stratifyingLabel,title,categoryLabels, testType):
    #Find z-scores for specified gene for each group, and store in list
    gene_expression_values = []
    for group in groups:
        group_columns = ['RefSeqID','symbol'] + list(set(group).intersection(z_scores.columns))
        group_z_scores = z_scores[group_columns]
        group_gene_expression_df = group_z_scores.loc[group_z_scores['symbol']==gene]
        group_gene_expression_values = group_gene_expression_df.iloc[0,2:].values.astype(float)
        gene_expression_values.append(group_gene_expression_values)
    
    #Update category labels to include sample size
    updated_categoryLabels = []
    for i in range(len(categoryLabels)):
        updated_categoryLabels.append(categoryLabels[i] + ' (n = ' + str(len(gene_expression_values[i])) + ')')
    
    #Make dataframe of gene expression values
    gene_expression_values_df = pd.DataFrame(index=range(max([len(i) for i in gene_expression_values])))
    for i in range(len(categoryLabels)):
        gene_expression_values_df[categoryLabels[i]] = pd.Series(gene_expression_values[i])
    
    #Violin plot comparing expression between groups
    ax1 = sns.violinplot(data=gene_expression_values)
    ax1.set_ylabel('z-scores')
    ax1.set_xticklabels(updated_categoryLabels)
    ax1.set_title(title)
    ax1, test_results = add_stat_annotation(ax1, data=gene_expression_values_df,
                    box_pairs=list(combinations(categoryLabels,2)),
                    test=testType, text_format='simple', loc='inside', verbose=2, comparisons_correction=None)
    plt.show()
    
    #Box plot comparing expression between groups
    ax2 = sns.boxplot(data=gene_expression_values,palette='Greys')
    for patch in ax2.artists:
        r, g, b, a = patch.get_facecolor()
        patch.set_facecolor((r, g, b, 0.7))
    ax2.set_ylabel('z-scores')
    ax2.set_xticklabels(updated_categoryLabels)
    ax2.set_title(title)
    ax2.set_ylim(-5,5)
    ax2, test_results = add_stat_annotation(ax2, data=gene_expression_values_df,
                    box_pairs=list(combinations(categoryLabels,2)),
                    test=testType, text_format='simple', loc='inside', verbose=2, comparisons_correction=None)
    plt.show()
    
    #Print median, mean, and 95% CI of z-scores for each group
    for i in range(len(gene_expression_values)):
        print(categoryLabels[i] + ": median = " + str(np.nanmedian(gene_expression_values[i]))
         + ", mean = " + str(np.nanmean(gene_expression_values[i])))
        

    
    
    

In [19]:
#This function compares the Event-Free Survival (EFS) between multiple groups
#Inputs: 2-D array containing lists of samples in each groups (output of functions above),
#title of graph, list of labels for each group
#Outputs: boxplot of EFS compared between groups, summary statistics
def compareEFS(groups, title, labels, testType):
    #Get EFS times for each group
    EFS = []
    for group in groups:
        group_EFS = clinical.loc[clinical['SampID'].isin(group)]['EFS Time'].astype(float).dropna()
        EFS.append(group_EFS)
    
    #Update  labels to include sample size
    updated_labels = []
    for i in range(len(labels)):
        updated_labels.append(labels[i] + ' (n = ' + str(len(EFS[i])) + ')')
        
    #Make dataframe of EFS times
    EFS_df = pd.DataFrame(index=range(max([len(i) for i in EFS])))
    for i in range(len(labels)):
        EFS_df[labels[i]] = pd.Series(EFS[i].values)
    
    #Boxplot comparing EFS time of each group
    ax = sns.boxplot(data=EFS)
    ax.set_ylabel('EFS (Days)')
    ax.set_xticklabels(updated_labels)
    ax.set_title(title)
    ax, test_results = add_stat_annotation(ax, data=EFS_df,
                    box_pairs=list(combinations(labels,2)),
                    test=testType, text_format='simple', loc='inside', verbose=2, comparisons_correction=None)
    plt.show()
    
    #Print median, mean, and 95% CI of z-scores for each group
    for i in range(len(EFS)):
        print(labels[i] + ": median = " + str(np.median(EFS[i]))
         + ", mean = " + str(np.mean(EFS[i]))
         + ", 95% CI = " + str(st.norm.interval(alpha=0.95, loc=np.mean(EFS[i]), scale=st.sem(EFS[i]))))
        
        

In [52]:
#This function plots a Kaplan Meier curve based on event-free survival of two groups
#Inputs: 2-D array containing lists of samples in two groups (output of functions above), string labels for each group
#Output: Kaplan Meier curve
#This function utilizes the kaplanmeier python package (make sure to cite): https://pypi.org/project/kaplanmeier/
def kaplanmeierEFS(groups, label1, label2):
    #Get EFS Times (in days) for each group
    group1_EFS = clinical.loc[clinical['SampID'].isin(groups[0])]['EFS Time'].astype(float)
    group2_EFS = clinical.loc[clinical['SampID'].isin(groups[1])]['EFS Time'].astype(float)
    
    #Get event status for each group
    group1_event_status = clinical.loc[clinical['SampID'].isin(groups[0])]['EFS Bin']
    group2_event_status = clinical.loc[clinical['SampID'].isin(groups[1])]['EFS Bin']
    
    #Create dataframe containing EFS Time and Event Status for all group1 samples
    group1_df = pd.DataFrame()
    group1_df['EFS_time'] = group1_EFS.values
    group1_df['Event_Status'] = group1_event_status.values
    group1_df['Group'] = label1
    
    #Create dataframe containing EFS Time and Event Status for all group2 samples
    group2_df = pd.DataFrame()
    group2_df['EFS_time'] = group2_EFS.values
    group2_df['Event_Status'] = group2_event_status.values
    group2_df['Group'] = label2
    
    #Combine dataframes
    combined_df = pd.concat([group1_df,group2_df]).dropna()
    
    #Use kaplanmeier Python package to plot Kaplan-Meier survival curve
    out = km.fit(combined_df['EFS_time'], combined_df['Event_Status'], combined_df['Group'])
    km.plot(out)
    
    #logrank_test
    results=logrank_test(group1_df['EFS_time'],group2_df['EFS_time'],event_observed_A=group1_df['Event_Status'], event_observed_B=group2_df['Event_Status'])
    results.print_summary()
    print('p-value = ' + str(results.p_value))
    
    

In [29]:
#This function plots a Kaplan Meier curve based on event-free survival of multiple groups
#Inputs: 2-D array containing lists of samples in each groups (output of functions above),
#title of graph, list of string labels for each group
#Output: Kaplan Meier curve
#This function utilizes the kaplanmeier python package (make sure to cite): https://pypi.org/project/kaplanmeier/
#This function is capable of comparing multiple groups, unlike original function
def kaplanmeierEFS2(groups, title, labels):
    #Get EFS times for each group
    EFS = []
    for group in groups:
        group_EFS = clinical.loc[clinical['SampID'].isin(group)]['EFS Time'].astype(float).dropna()
        group_EFS = group_EFS/365
        EFS.append(group_EFS)
    
    #Get event status for each group
    event_status = []
    for group in groups:
        event_num = clinical.loc[clinical['SampID'].isin(group)]['EFS Bin'].astype(float)
        event_status.append(event_num)
    
    #Create dataframe containing EFS Time and Event Status for each group
    group_dfs = []
    for i in range(len(groups)):
        group_df = pd.DataFrame()
        group_df['EFS_time'] = EFS[i]
        group_df['Event_Status'] = event_status[i]
        group_df['Group'] = labels[i]
        group_dfs.append(group_df)
    
    #Combine dataframes
    combined_df = pd.concat(group_dfs).dropna()
    
    #Use kaplanmeier Python package to plot Kaplan-Meier survival curve
    linestyles = [':','--','-']
    for i in range(len(groups)):
        kmf = KaplanMeierFitter(alpha=1)
        kmf.fit(group_dfs[i]['EFS_time'], group_dfs[i]['Event_Status'], label=labels[i])
        ax = kmf.plot(color='black',linestyle=linestyles[i])
        #Print median survival time and probability of 5yr survival
        print(labels[i] + ' median survival time = ' + str(kmf.median_survival_time_))
        print(labels[i] + ' probability of 5yr EFS = ' + str(kmf.predict(5)))
    ax.set_title(title)
    ax.set_ylabel('EFS proportion')
    ax.set_xlabel('Years')
    ax.set_ylim([0,1])
    ax.legend(loc='lower left')
    plt.show()
    
    #logrank_test
    for combo in list(combinations(labels,2)):
        group1=combined_df[combined_df['Group']==combo[0]]
        group2=combined_df[combined_df['Group']==combo[1]]
        T=group1['EFS_time']
        E=group1['Event_Status']
        T1=group2['EFS_time']
        E1=group2['Event_Status']
        results=logrank_test(T,T1,event_observed_A=E, event_observed_B=E1)
        print('Log Rank Test between ' + combo[0] + ' and ' + combo[1])
        results.print_summary()
        print('p-value = ' + str(results.p_value))
    
    

In [154]:
def geneScatter(gene1, gene2, z_scores=z_scores):
    gene1_z_scores = z_scores.loc[z_scores['symbol']==gene1].iloc[:,2:].transpose().iloc[:,0]
    gene2_z_scores = z_scores.loc[z_scores['symbol']==gene2].iloc[:,2:].transpose().iloc[:,0]
    merged_z_scores = pd.concat([gene1_z_scores,gene2_z_scores],axis=1)
    merged_z_scores = merged_z_scores.dropna()
    
    plt.scatter(merged_z_scores.iloc[:,0],merged_z_scores.iloc[:,1])
    plt.xlabel(gene1 + ' z-score')
    plt.ylabel(gene2 + ' z-score')
    m,b = np.polyfit(merged_z_scores.iloc[:,0].values,merged_z_scores.iloc[:,1].values,1)
    plt.plot(merged_z_scores.iloc[:,0], m*(merged_z_scores.iloc[:,0].values) + b, 'orange')
    plt.show()
    
    corr,pvalue = st.pearsonr(merged_z_scores.iloc[:,0].tolist(),merged_z_scores.iloc[:,1].tolist())
    print('Pearson correlation coefficient = ' + str(corr))
    print('p-value = ' + str(pvalue))
